#PNEUMONIA DETECTION USING CNN


In this project we are going to be building a convolutional neural network to recognize whether the patient is healthy (label 0) or has pneumonia (label 1).
The dataset used can be downloaded [here](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/download):

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path_training_set = "/content/drive/MyDrive/chest_xray/train"
path_test_set = "/content/drive/MyDrive/chest_xray/train"

img_size = 64
batch_size = 32

datagen_train = ImageDataGenerator(horizontal_flip=True)

train_generator = datagen_train.flow_from_directory(path_training_set,
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

datagen_validation = ImageDataGenerator(horizontal_flip=True)

validation_generator = datagen_validation.flow_from_directory(path_test_set,
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 5190 images belonging to 2 classes.
Found 5190 images belonging to 2 classes.


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32,(7,7),padding="same", strides=2, 
                         input_shape = (img_size,img_size,1)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Activation("relu"),
  tf.keras.layers.MaxPooling2D(3,3),

  tf.keras.layers.Conv2D(64,(1,1),padding="same", strides=1),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Activation("relu"),

  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(32),
  tf.keras.layers.Dense(2,activation="softmax")
])


In [4]:

steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=1, # only output a limited amount of text
                                            save_best_only=True) # save only the best model to file
model.compile(loss = "binary_crossentropy",
              metrics = ["accuracy"],
              optimizer = tf.keras.optimizers.Adam(0.01))

history = model.fit(
    x= train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data = validation_generator,
    validation_steps = validation_steps,
    callbacks=create_model_checkpoint(model_name=model.name),
)

Epoch 1/20
162/162 [==============================] - ETA: 0s - loss: 1.1168 - accuracy: 0.9161
Epoch 00001: val_loss improved from inf to 0.29307, saving model to model_experiments/sequential
INFO:tensorflow:Assets written to: model_experiments/sequential/assets
162/162 [==============================] - 125s 737ms/step - loss: 1.1168 - accuracy: 0.9161 - val_loss: 0.2931 - val_accuracy: 0.9389
Epoch 2/20
162/162 [==============================] - ETA: 0s - loss: 0.1686 - accuracy: 0.9583
Epoch 00002: val_loss improved from 0.29307 to 0.18688, saving model to model_experiments/sequential
INFO:tensorflow:Assets written to: model_experiments/sequential/assets
162/162 [==============================] - 118s 728ms/step - loss: 0.1686 - accuracy: 0.9583 - val_loss: 0.1869 - val_accuracy: 0.9363
Epoch 3/20
162/162 [==============================] - ETA: 0s - loss: 0.1145 - accuracy: 0.9601
Epoch 00003: val_loss improved from 0.18688 to 0.15773, saving model to model_experiments/sequential
I

In [5]:
model = tf.keras.models.load_model("model_experiments/sequential_1")
preds = model.predict(train_generator)

In [10]:
class_mapping = train_generator.class_indices
class_mapping_test = validation_generator.class_indices
print(class_mapping_test)

{'NORMAL': 0, 'PNEUMONIA': 1}


In [6]:
import glob
import cv2
test_dir = "/content/drive/MyDrive/test_images_pneumonia"
data_path = os.path.join(test_dir,'*g')
test = []
files = glob.glob(data_path)
for f1 in files:  
    img = cv2.imread(f1, cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(img, (img_size,img_size))
    resized_image = np.expand_dims(resized_image,-1)
    test.append(resized_image)
    print(f1)

test = np.asarray(test)
print(test.shape)



/content/drive/MyDrive/test_images_pneumonia/person5_bacteria_15.jpeg
/content/drive/MyDrive/test_images_pneumonia/person5_bacteria_16.jpeg
/content/drive/MyDrive/test_images_pneumonia/person5_bacteria_19.jpeg
/content/drive/MyDrive/test_images_pneumonia/person5_bacteria_17.jpeg
/content/drive/MyDrive/test_images_pneumonia/person6_bacteria_22.jpeg
/content/drive/MyDrive/test_images_pneumonia/person7_bacteria_24.jpeg
/content/drive/MyDrive/test_images_pneumonia/IM-0122-0001.jpeg
/content/drive/MyDrive/test_images_pneumonia/IM-0125-0001.jpeg
/content/drive/MyDrive/test_images_pneumonia/IM-0115-0001.jpeg
/content/drive/MyDrive/test_images_pneumonia/IM-0135-0001.jpeg
/content/drive/MyDrive/test_images_pneumonia/IM-0127-0001.jpeg
/content/drive/MyDrive/test_images_pneumonia/IM-0128-0001.jpeg
/content/drive/MyDrive/test_images_pneumonia/IM-0131-0001.jpeg
/content/drive/MyDrive/test_images_pneumonia/IM-0117-0001.jpeg
/content/drive/MyDrive/test_images_pneumonia/IM-0119-0001.jpeg
/content/driv

In [7]:
test_preds = model.predict(test)
class_preds = np.argmax(test_preds, axis=-1)
print(class_preds)

[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]


**The accuracy achieved is above 97%.
The error on the previous 26 images is 3.84% (1 misclassified image out of 26).**